#  Notebook Tutorial on an Empty SAIL-ON Detector

In this example, it will show you an example of an empty detector which can be passed into the SAIL-ON-CLIENT protocol via the SAIL-ON Launcher.  

In [1]:
%load_ext autoreload
%autoreload 2

# Setup Logging to print to the notebook
import logging
import colorlog

handler = colorlog.StreamHandler()
handler.setFormatter(
    colorlog.ColoredFormatter(
        fmt='[%(cyan)s%(asctime)s%(reset)s][%(blue)s%(name)s%(reset)s]'
            '[%(log_color)s%(levelname)s%(reset)s] - %(message)s',
        log_colors={
            'DEBUG': 'purple',
            'INFO': 'green',
            'WARNING': 'yellow',
            'ERROR': 'red',
            'CRITICAL': 'red,bg_white',
        },
    ),
)

log = colorlog.getLogger('notebook')
log.setLevel(logging.DEBUG)
log.addHandler(handler)

### Initalize the launcher with hydra to load the config

Note: this config will define the save directory so make sure to edit any configuration values before running the code

In [2]:
from hydra.experimental import initialize, compose
from omegaconf import OmegaConf
import yaml

# Initialize Hydra to get the default config.  Overrides of `problem=mock_ond_config` will be selecting 
#     the 'mock_ond_config' from the folder '<sailon_tinker_launcher_root>/configs/problem/mock_ond_config.yaml'  
with initialize(config_path="../configs"):
    cfg = compose(config_name="h_config", overrides=['problem=mock_ond_config'])
    print(cfg)
    
# You can also update the parameters here but do it before you convert to a dict (next step).  
#   Here we update the config with a new novelty class detector name (to match the class in the next cell).
cfg.problem.novelty_detector_class = 'MyDetector'

# Turn the omegaconfig to a dict object and print it nicely
config = OmegaConf.to_container(cfg['problem'])
print(yaml.dump(config, indent=4))

{'problem': {'protocol': 'ond', 'workdir': '/home/chris/code/sailon_tinker_launcher/workdir', 'harness': 'local', 'domain': 'image_classification', 'test_ids': ['OND.1.1.1234'], 'novelty_detector_class': 'MockDetector'}}
domain: image_classification
harness: local
novelty_detector_class: MyDetector
protocol: ond
test_ids:
- OND.1.1.1234
workdir: /home/chris/code/sailon_tinker_launcher/workdir



### Create Python Class to Run for the experiment.

This is using an empty detector which just says it's name (similar to a pokemon that can't fight).  This is actually better since it also said the step it's running.  

In [3]:
from sailon_tinker_launcher.deprecated_tinker.basealgorithm import BaseAlgorithm
from evm_based_novelty_detector.condda_12 import condda_without_redlight
from typing import Dict, Any, Tuple


class MyDetector(BaseAlgorithm):
    """Mock Detector for testing image classification protocols."""

    def __init__(self, toolset: Dict) -> None:
        """
        Detector constructor.

        Args:
            toolset (dict): Dictionary containing parameters for the constructor
        """
        BaseAlgorithm.__init__(self, toolset)
        self.step_dict: Dict[str, Callable] = {
            "Initialize": self._initialize,
            "FeatureExtraction": self._feature_extraction,
            "WorldDetection": self._world_detection,
            "NoveltyClassification": self._novelty_classification,
            "NoveltyAdaption": self._novelty_adaption,
            "NoveltyCharacterization": self._novelty_characterization,
        }

    def execute(self, toolset: Dict, step_descriptor: str) -> Any:
        """
        Execute method used by the protocol to run different steps associated with the algorithm.

        Args:
            toolset (dict): Dictionary containing parameters for different steps
            step_descriptor (str): Name of the step
        """
        log.info(f"Executing Step: {step_descriptor}")
        return self.step_dict[step_descriptor](toolset)

    def _initialize(self, toolset: Dict) -> None:
        """
        Algorithm Initialization.

        Args:
            toolset (dict): Dictionary containing parameters for different steps

        Return:
            None
        """
        log.info('Initialize')

    def _feature_extraction(
        self, toolset: Dict
    ) -> Tuple[Dict[str, Any], Dict[str, Any]]:
        """
        Feature extraction step for the algorithm.

        Args:
            toolset (dict): Dictionary containing parameters for different steps

        Return:
            Tuple of dictionary
        """
        self.dataset = toolset["dataset"]
        log.info('Feature Extraction')
        return {}, {}

    def _world_detection(self, toolset: str) -> str:
        """
        Detect change in world ( Novelty has been introduced ).

        Args:
            toolset (dict): Dictionary containing parameters for different steps

        Return:
            path to csv file containing the results for change in world
        """
        log.info('World Detection')
        return self.dataset

    def _novelty_classification(self, toolset: str) -> str:
        """
        Classify data provided in known classes and unknown class.

        Args:
            toolset (dict): Dictionary containing parameters for different steps

        Return:
            path to csv file containing the results for novelty classification step
        """
        log.info('Novelty Classification')
        return self.dataset

    def _novelty_adaption(self, toolset: str) -> None:
        """
        Update models based on novelty classification and characterization.

        Args:
            toolset (dict): Dictionary containing parameters for different steps

        Return:
            None
        """
        log.info('Novelty Adaption')

    def _novelty_characterization(self, toolset: str) -> str:
        """
        Characterize novelty by clustering different novel samples.

        Args:
            toolset (dict): Dictionary containing parameters for different steps

        Return:
            path to csv file containing the results for novelty characterization step
        """
        log.info('Novelty Characterization')
        return self.dataset

### launch the detector in the sailon system

The extra plugin parameter takes a dictionary which allows you to add your detector to the list of detectors.  The name of the detector in the config needs to match the key in the dictionary.  See config above for more details.  

In [4]:
from sailon_tinker_launcher.main import LaunchSailonProtocol


launch_protocol = LaunchSailonProtocol()
launch_protocol.run_protocol(config, extra_plugins={'MyDetector': MyDetector})


Using backend: pytorch
[2021-02-03 11:33:29,330][notebook][INFO] - Executing Step: Initialize
[2021-02-03 11:33:29,331][notebook][INFO] - Initialize
[2021-02-03 11:33:29,334][notebook][INFO] - Executing Step: FeatureExtraction
[2021-02-03 11:33:29,335][notebook][INFO] - Feature Extraction
[2021-02-03 11:33:29,336][notebook][INFO] - Executing Step: WorldDetection
[2021-02-03 11:33:29,337][notebook][INFO] - World Detection
[2021-02-03 11:33:29,338][notebook][INFO] - Executing Step: NoveltyClassification
[2021-02-03 11:33:29,340][notebook][INFO] - Novelty Classification
[2021-02-03 11:33:29,345][notebook][INFO] - Executing Step: FeatureExtraction
[2021-02-03 11:33:29,346][notebook][INFO] - Feature Extraction
[2021-02-03 11:33:29,347][notebook][INFO] - Executing Step: WorldDetection
[2021-02-03 11:33:29,348][notebook][INFO] - World Detection
[2021-02-03 11:33:29,349][notebook][INFO] - Executing Step: NoveltyClassification
[2021-02-03 11:33:29,350][notebook][INFO] - Novelty Classification
[